# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 2 2023 10:27AM,256492,10,Enova-11304,Emerginnova,Released
1,Feb 2 2023 10:25AM,256491,19,9107544,VITABIOTICS USA INC.,Released
2,Feb 2 2023 10:15AM,256490,10,SOTAH0000898,"Nextsource Biotechnology, LLC",Released
3,Feb 2 2023 10:12AM,256488,10,PRF-51370,Bio-PRF,Released
4,Feb 2 2023 10:12AM,256488,10,PRF-51387,Bio-PRF,Released
5,Feb 2 2023 10:12AM,256488,10,PRF-51399,Bio-PRF,Released
6,Feb 2 2023 10:12AM,256488,10,PRF-51416,Bio-PRF,Released
7,Feb 2 2023 10:12AM,256488,10,PRF-51425,Bio-PRF,Released
8,Feb 2 2023 10:12AM,256488,10,PRF-51491,Bio-PRF,Released
9,Feb 2 2023 10:12AM,256488,10,PRF-51531,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,256488,Released,7
33,256489,Released,18
34,256490,Released,1
35,256491,Released,1
36,256492,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256488,NaN,NaN,7.0
256489,NaN,NaN,18.0
256490,NaN,NaN,1.0
256491,NaN,NaN,1.0
256492,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256392,0.0,0.0,1.0
256401,0.0,1.0,0.0
256413,16.0,10.0,0.0
256414,0.0,0.0,1.0
256418,3.0,2.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256392,0,0,1
256401,0,1,0
256413,16,10,0
256414,0,0,1
256418,3,2,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256392,0,0,1
1,256401,0,1,0
2,256413,16,10,0
3,256414,0,0,1
4,256418,3,2,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256392,,,1
1,256401,,1,
2,256413,16,10,
3,256414,,,1
4,256418,3,2,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 2 2023 10:27AM,256492,10,Emerginnova
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC"
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF
10,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.
28,Feb 2 2023 10:03AM,256469,15,"Mizner Bioscience, LLC"
29,Feb 2 2023 10:03AM,256470,15,"Virtus Pharmaceuticals, LLC"
32,Feb 2 2023 10:00AM,256471,15,"Person & Covey, Inc."
45,Feb 2 2023 9:58AM,256487,22,"NBTY Global, Inc."
46,Feb 2 2023 9:56AM,256486,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 2 2023 10:27AM,256492,10,Emerginnova,,,1
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.,,,1
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC",,,1
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF,,,7
4,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.,,,18
5,Feb 2 2023 10:03AM,256469,15,"Mizner Bioscience, LLC",,,1
6,Feb 2 2023 10:03AM,256470,15,"Virtus Pharmaceuticals, LLC",,,3
7,Feb 2 2023 10:00AM,256471,15,"Person & Covey, Inc.",,,13
8,Feb 2 2023 9:58AM,256487,22,"NBTY Global, Inc.",,,1
9,Feb 2 2023 9:56AM,256486,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 10:27AM,256492,10,Emerginnova,1,,
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.,1,,
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC",1,,
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF,7,,
4,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.,18,,
5,Feb 2 2023 10:03AM,256469,15,"Mizner Bioscience, LLC",1,,
6,Feb 2 2023 10:03AM,256470,15,"Virtus Pharmaceuticals, LLC",3,,
7,Feb 2 2023 10:00AM,256471,15,"Person & Covey, Inc.",13,,
8,Feb 2 2023 9:58AM,256487,22,"NBTY Global, Inc.",1,,
9,Feb 2 2023 9:56AM,256486,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 10:27AM,256492,10,Emerginnova,1,,
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.,1,,
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC",1,,
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF,7,,
4,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.,18,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 10:27AM,256492,10,Emerginnova,1.0,NaN,NaN
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.,1.0,NaN,NaN
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF,7.0,NaN,NaN
4,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.,18.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 10:27AM,256492,10,Emerginnova,1.0,0.0,0.0
1,Feb 2 2023 10:25AM,256491,19,VITABIOTICS USA INC.,1.0,0.0,0.0
2,Feb 2 2023 10:15AM,256490,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Feb 2 2023 10:12AM,256488,10,Bio-PRF,7.0,0.0,0.0
4,Feb 2 2023 10:09AM,256489,10,Eywa Pharma Inc.,18.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3077526,58.0,2.0,6.0
12,769271,2.0,1.0,0.0
15,1025878,52.0,7.0,0.0
19,1795122,3.0,13.0,16.0
21,1282321,5.0,0.0,0.0
22,256487,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3077526,58.0,2.0,6.0
1,12,769271,2.0,1.0,0.0
2,15,1025878,52.0,7.0,0.0
3,19,1795122,3.0,13.0,16.0
4,21,1282321,5.0,0.0,0.0
5,22,256487,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,2.0,6.0
1,12,2.0,1.0,0.0
2,15,52.0,7.0,0.0
3,19,3.0,13.0,16.0
4,21,5.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,12,Released,2.0
2,15,Released,52.0
3,19,Released,3.0
4,21,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,6.0,0.0,0.0,16.0,0.0,0.0
Executing,2.0,1.0,7.0,13.0,0.0,0.0
Released,58.0,2.0,52.0,3.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,6.0,0.0,0.0,16.0,0.0,0.0
1,Executing,2.0,1.0,7.0,13.0,0.0,0.0
2,Released,58.0,2.0,52.0,3.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,6.0,0.0,0.0,16.0,0.0,0.0
1,Executing,2.0,1.0,7.0,13.0,0.0,0.0
2,Released,58.0,2.0,52.0,3.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()